In [10]:
import pandas as pd
import numpy as np
import random

# File operations
from copy import deepcopy
import glob
import os
from pathlib import Path

# Stats
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr, wilcoxon, mannwhitneyu
import statsmodels.stats.multitest

# Neuromaps
import neuromaps
from neuromaps.parcellate import Parcellater
from neuromaps import datasets, images, nulls, resampling
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation, fetch_fslr
from enigmatoolbox.datasets import load_sc, load_fc
from nilearn.image import resample_to_img

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(cowplot)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

# Load in the data and filter to Pearson correlation

In [31]:
data_path = "/Users/abry4213/data/HCP100/"
github_data_path = "/Users/abry4213/github/Homotopic_FC/data/"

#### GROUP-AVERAGED FUNCTIONAL CONNECTOME FROM ENIGMA #### 
# Load cortico-cortical functional connectivity data
fc_ctx, fc_ctx_labels, _, _ = load_fc(parcellation='aparc')

# Read in preprocessed Pearson-based functional connectivity data from ENIGMA
HCP_avg_Pearson_FC = (pd.DataFrame(fc_ctx, columns=fc_ctx_labels)
                      .assign(Brain_Region_From = fc_ctx_labels)
                      .melt(id_vars=["Brain_Region_From"], 
                             var_name="Brain_Region_To", 
                             value_name="Mean_Pearson_R")
                      .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.replace("L_", "").str.replace("R_", ""),
                              Base_Region_To = lambda x: x['Brain_Region_To'].str.replace("L_", "").str.replace("R_", ""),
                              Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.startswith("L_"), "Left", "Right"),
                              Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.startswith("L_"), "Left", "Right"))
)
pearson_homotopic_res_mean = (HCP_avg_Pearson_FC
                              .query("Base_Region_From == Base_Region_To")
                              .groupby(["Base_Region_From"])['Mean_Pearson_R']
                              .mean() 
                              .reset_index()
                              .rename(columns={"Base_Region_From": "Base_Region"})
                              )

sc_ctx, sc_ctx_labels, _, _ = load_sc(parcellation='aparc')

HCP_avg_Pearson_SC = (pd.DataFrame(sc_ctx, columns=sc_ctx_labels)
                      .assign(Brain_Region_From = sc_ctx_labels)
                      .melt(id_vars=["Brain_Region_From"], 
                             var_name="Brain_Region_To", 
                             value_name="Mean_Num_Streamlines")
                      .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.replace("L_", "").str.replace("R_", ""),
                              Base_Region_To = lambda x: x['Brain_Region_To'].str.replace("L_", "").str.replace("R_", ""),
                              Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.startswith("L_"), "Left", "Right"),
                              Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.startswith("L_"), "Left", "Right"))
)

SC_homotopic_res_mean = (HCP_avg_Pearson_SC
                            .query("Base_Region_From == Base_Region_To")
                            .groupby(["Base_Region_From"])['Mean_Num_Streamlines']
                            .mean() 
                            .reset_index()
                            .rename(columns={"Base_Region_From": "Base_Region"})
                            .rename(columns={"Mean_Num_Streamlines": "Value"})
                            .assign(Category = "Microstructure", Description = "Log-SC")
                            )

In [54]:
%%R -i pearson_homotopic_res_mean

### Pearson FC in brain ###
pearson_homotopic_res_mean %>%
  ungroup() %>%
  mutate(label = glue("lh_{Base_Region}")) %>%
  left_join(., as_tibble(dk)) %>%
  ggseg(atlas = dk, mapping = aes(fill = Mean_Pearson_R),
        position = "stacked", colour = "black", hemisphere="left",
        linewidth=0.5) +
  theme_void() +
  labs(fill = "Mean Pearson R") +
      theme(plot.title = element_blank(),
            legend.key.width  = unit(3, "lines"),
            legend.key.height  = unit(0.75, "lines"),
            legend.position = "bottom") +
            guides(fill = guide_colorbar(title.position="top", title.hjust=0.5)) +
    scale_fill_gradient(low='white', high='#A469E6', na.value='white')
ggsave("../plots/measure_maps/Mean_LR_averaged_Pearson_R_ggseg.svg", width = 4, height = 3)

Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'


## Is the magnitude of homotopic Pearson FC across brain regions tied to (1) Euclidean distance between the two regions and/or (2) the number of streamlines between those two regions on average?

### Calculate Euclidean, geodesic, and fiber connectivity distances between each region--region pair

In [32]:
# Read in brain region lookup table
brain_region_info = pd.read_csv("../data/Brain_Region_Info.csv")
fsaverage_data_base_path = "/Users/abry4213/data/neuroimaging_atlases/surfaces/DesikanKilliany/fsaverage"

all_centroid_res_list = []
for hemi in ["lh", "rh"]:

    # Load annotation file (parcellation)
    annot_path = f"{fsaverage_data_base_path}/{hemi}.aparc.annot"
    labels, ctab, names = fs.read_annot(annot_path)

    # Load label file (vertex coordinates)
    label_path = f"{fsaverage_data_base_path}/{hemi}.aparc.label"
    vertices = fs.read_label(label_path)

    # Load the FreeSurfer surface (to get vertex coordinates)
    pial_path = f"{fsaverage_data_base_path}/{hemi}.pial"
    surf = fs.read_geometry(pial_path)
    coords, faces = surf
    cort = np.sort(fs.read_label(f"{fsaverage_data_base_path}/{hemi}.cortex.label"))

    # Create a dictionary to store centroids
    roi_centroids = {}

    # Compute centroids for each ROI
    for i, roi_name in enumerate(names):
        roi_vertices = np.where(labels == i)[0]  # Get vertex indices in this ROI
        if len(roi_vertices) > 0:
            roi_centroids[roi_name] = coords[roi_vertices].mean(axis=0)  # Mean position

    # Convert centroids to DataFrame
    centroid_df = pd.DataFrame(roi_centroids).T
    centroid_df.columns = ["x", "y", "z"]

    # Convert names to strings, dropping corpuscallosum
    names_dec = [name.decode() for name in names if name.decode() not in ["corpuscallosum"]]

    # If centroid_df has 34 rows, drop 'unknown' from names
    if centroid_df.shape[0] == 34:
        names_dec = [name for name in names_dec if name != "unknown"]

    # Add region names and hemisphere
    centroid_df = (centroid_df.reset_index(drop=True).assign(Base_Region=names_dec, 
                                                             Hemisphere=np.where(hemi == "lh", "Left", "Right"))
                                                     .assign(Brain_Region = lambda x: hemi + '-' + x['Base_Region']))

    # Append to list
    all_centroid_res_list.append(centroid_df)

# Concatenate the results
all_centroid_res = pd.concat(all_centroid_res_list).query("Base_Region != 'unknown'")

# Compute Euclidean distances between all ROI centroids
dist_matrix = cdist(all_centroid_res[['x', 'y', 'z']], all_centroid_res[['x', 'y', 'z']], metric="euclidean")

# Convert to DataFrame
euclidean_dist_df = (pd.DataFrame(dist_matrix, index=all_centroid_res['Brain_Region'], 
                        columns=all_centroid_res['Brain_Region'])
                        .reset_index()
                        .rename(columns={"Brain_Region": "Brain_Region_From"})
                        .melt(id_vars='Brain_Region_From', var_name='Brain_Region_To', value_name='Euclidean_Distance')
                        .query("Brain_Region_From != Brain_Region_To")
                        .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.split('-').str[1],
                                Base_Region_To = lambda x: x['Brain_Region_To'].str.split('-').str[1],
                                Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.split('-').str[0]=='lh', 'Left', 'Right'),
                                Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.split('-').str[0]=='lh', 'Left', 'Right'))
                        )

# Filter to just the homotopic regions
homotopic_euclidean_dist_df = (euclidean_dist_df.query("Base_Region_From == Base_Region_To").drop_duplicates(subset=['Base_Region_To', 'Euclidean_Distance'])
                               .filter(['Base_Region_To', 'Euclidean_Distance'])
                               .rename(columns={"Base_Region_To": "Base_Region",
                                                "Euclidean_Distance": "Value"})
                               .assign(Category = "Microstructure", Description = "Euclidean Distance")
                               )

# Merge the results
structural_data = (pd.concat([homotopic_euclidean_dist_df, SC_homotopic_res_mean])
                           .merge(pearson_homotopic_res_mean, on="Base_Region", how="left"))

In [51]:
%%R -i structural_data

colormap <- setNames(c("#5c9335", "#10b8e8"), c('Euclidean Distance', 'Log-SC'))

all_scatter_plots_list <- list()

for (desc in unique(structural_data$Description)) {
      desc_color <- colormap[desc]

      desc_p_scatter <- structural_data %>% 
        filter(Description == desc) %>%
        ggplot(aes(x=Value, y=Mean_Pearson_R, fill=Value)) +
        geom_point(color='black', shape=21, size=3) +
        scale_fill_gradient(low="white", high=desc_color, na.value="white") +
        geom_smooth(method="lm", se=FALSE, color='black') +
        ggtitle(desc) +
        theme(legend.position="none") +
        ggpubr::stat_cor(method="spearman", cor.coef.name="rho", size=5, aes(label = ..r.label..)) +
        xlab("Value") +
        ylab("Mean Pearson R") +
        theme(strip.background = element_blank(),
            plot.title = element_text(hjust=0.5, face='bold', size=16),
            strip.text = element_text(face='bold', size=14))

      all_scatter_plots_list[[desc]] <- desc_p_scatter
}

wrap_plots(all_scatter_plots_list, ncol=1)

ggsave("../plots/measure_maps/Mean_Pearson_R_magnitude_vs_physical_distance.svg", width=2.7, height=5.25, dpi=300, units="in")

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In addition: Warning messages:
1: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
2: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
3: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
4: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping

### Plot the SC/Euclidean distance per region in the brain

In [40]:
%%R -i structural_data

all_ggseg_plots_list <- list()

colormap <- setNames(c("#5c9335", "#10b8e8"), c('Euclidean Distance', 'Log-SC'))

for (desc in unique(structural_data$Description)) {
      desc_color <- colormap[desc]

      desc_p <- structural_data %>%
            filter(Description==desc) %>% 
            mutate(label = glue("lh_{Base_Region}")) %>%
            left_join(., as_tibble(dk)) %>%
            ggseg(atlas = dk, mapping = aes(fill = Value),
                  position = "stacked", colour = "black", hemisphere="left",
                  linewidth=0.5) +
            theme_void() +
            labs(fill = desc) +
            scale_fill_gradient(low="white", high=desc_color, na.value="white")+
            theme(plot.title = element_blank(),
            legend.key.width  = unit(3, "lines"),
            legend.key.height  = unit(0.75, "lines"),
            legend.position = "bottom") +
            guides(fill = guide_colorbar(title.position="top", title.hjust=0.5))

      all_ggseg_plots_list[[desc]] <- desc_p
}

wrap_plots(all_ggseg_plots_list, ncol=1)
ggsave("../plots/measure_maps/structural_data_ggseg_res.svg", width=4, height=4, dpi=300, units="in")


Joining with `by = join_by(label)`
Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'


## Is mean homotopic FC correlated with region volume?

In [7]:
# Load in regional volumes
regional_volumes = pd.read_csv(f"{data_path}/raw_data/structural_MRI/HCP100_all_dk_volumes.csv")
regional_volumes_mean = (regional_volumes
                         .groupby('Region')['Volume']
                         .agg(['mean', 'std'])
                         .reset_index()
                         .rename(columns={"Region": "Region_Index", "mean": "Mean_Volume", "std": "SD_Volume"})
                         .assign(Region_Index = lambda x: x.Region_Index.astype(int))
                         .merge(brain_region_info, on="Region_Index", how="left")
)

regional_volumes_mean_hemi_avg = (regional_volumes
    .rename(columns={"Region": "Region_Index"})
    .merge(brain_region_info, on="Region_Index", how="left")
    .groupby('Base_Region')['Volume']
    .agg(['mean', 'std'])
    .reset_index()
    .rename(columns={"mean": "Mean_Volume", "std": "SD_Volume"})
    .merge(pearson_homotopic_res_mean, on="Base_Region", how="left")
)

# Fit a Spearman correlation
volume_pearson_corr = spearmanr(regional_volumes_mean_hemi_avg['Mean_Volume'], regional_volumes_mean_hemi_avg['Mean_Pearson_R'])
volume_pearson_corr

SignificanceResult(statistic=np.float64(0.0985485103132162), pvalue=np.float64(0.5792431104724365))

In [47]:
%%R -i regional_volumes_mean_hemi_avg

regional_volumes_mean_hemi_avg %>%
    ggplot(aes(x=Mean_Volume, y=Mean_Pearson_R, fill=Mean_Volume)) +
    geom_point(color='black', shape=21, size=4) +
      scale_fill_gradient(low="white", high="#F1D201", na.value="white")+
    geom_smooth(method="lm", se=FALSE, color='black') +
    ggpubr::stat_cor(method="spearman", cor.coef.name="rho", size=5, aes(label = ..r.label..)) +
    xlab("Mean Volume") +
    ylab("Mean Pearson R") +
    theme(legend.position="none",
          axis.text = element_text(size=16),
          axis.title = element_text(size=16))
ggsave("../plots/measure_maps/Mean_Pearson_R_magnitude_vs_volume.svg", width=3.5, height=3.2, dpi=300, units="in")


`geom_smooth()` using formula = 'y ~ x'


In addition: Warning messages:
1: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
2: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 


In [44]:
%%R -i regional_volumes_mean_hemi_avg

regional_volumes_mean_hemi_avg %>%
      mutate(label = glue("lh_{Base_Region}")) %>%
      left_join(., as_tibble(dk)) %>%
      ggseg(atlas = dk, mapping = aes(fill = Mean_Volume),
            position = "stacked", colour = "black", hemisphere="left",
            linewidth=0.5) +
      theme_void() +
      labs(fill = "Mean volume") +
      scale_fill_gradient(low="white", high="#F1D201", na.value="white")+
      theme(plot.title = element_blank(),
            legend.key.width  = unit(3, "lines"),
            legend.key.height  = unit(0.75, "lines"),
            legend.position = "bottom") +
      guides(fill = guide_colorbar(title.position="top", title.hjust=0.5))

ggsave("../plots/measure_maps/Mean_Pearson_R_magnitude_vs_volume_ggseg.svg", width=4, height=2, dpi=300, units="in")


Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'


## Associations with vascular density

In [13]:
if not os.path.isfile(f"{github_data_path}/Arterial_Venous_Mean_Density_aparc.csv"):

    # aparc + aseg volume
    ROI_image = nib.load("/Users/abry4213/data/neuroimaging_atlases/volumes/freesurfer_segmentations/mni152_space/atlas-desikankilliany.nii.gz")
    ROI_lookup = pd.read_csv("/Users/abry4213/data/neuroimaging_atlases/volumes/freesurfer_segmentations/mni152_space/atlas-desikankilliany.csv")

    # Read in vasculature volumes (arterial and venous)
    arterial_image = nib.load(f"{github_data_path}/vasculature/mean_Ved_ToF_filt.nii.gz")
    venous_image = nib.load(f"{github_data_path}/vasculature/mean_Ved_swi_filt.nii.gz")

    # Downsample the segmentation volume to match the vasculature images
    ROI_image_resampled = resample_to_img(ROI_image, arterial_image, interpolation='nearest')
    ROI_data_resampled = ROI_image_resampled.get_fdata().flatten()

    # Extract data arrays
    venous_data = venous_image.get_fdata().flatten()
    arterial_data = arterial_image.get_fdata().flatten()

    ROI_indices = [int(i) for i in np.unique(ROI_data_resampled) if i != 0]

    regional_vascular_data_list = [] 

    for this_index in ROI_indices:
        region_info = ROI_lookup.query("id == @this_index")[['id', 'label', 'hemisphere']]

        arterial_roi_data = np.mean(arterial_data[ROI_data_resampled == this_index])
        venous_roi_data = np.mean(venous_data[ROI_data_resampled == this_index])

        ROI_vascular_df = pd.DataFrame({"id": this_index,
                                        "Arterial_Vascular": arterial_roi_data,
                                        "Venous_Vascular": venous_roi_data}, index=[0]).merge(region_info, on='id', how='left')
        
        regional_vascular_data_list.append(ROI_vascular_df)

    regional_vascular_data = (pd.concat(regional_vascular_data_list, ignore_index=True)
                            .rename(columns={"label": "Base_Region",
                                            'hemisphere': "Hemisphere"})
    )

    # Save to CSV
    regional_vascular_data.to_csv(f"{github_data_path}/Arterial_Venous_Mean_Density_aparc.csv", index=False)

else: 
    regional_vascular_data = pd.read_csv(f"{github_data_path}/Arterial_Venous_Mean_Density_aparc.csv")

# Take the mean of left/right
regional_vascular_data = regional_vascular_data.groupby(['Base_Region'])[['Arterial_Vascular', 'Venous_Vascular']].mean().reset_index()

In [24]:
%%R -i regional_vascular_data

### Arterial density brain ###
regional_vascular_data %>%
  ungroup() %>%
  mutate(label = glue("lh_{Base_Region}")) %>%
  left_join(., as_tibble(dk)) %>%
  ggseg(atlas = dk, mapping = aes(fill = Arterial_Vascular),
        position = "stacked", colour = "black", hemisphere="left",
        linewidth=0.5) +
  theme_void() +
  labs(fill = "Mean arterial density") +
  scale_fill_gradient(low = "white", high = "#e24528", na.value="white") +
  theme(plot.title = element_blank(),
        legend.key.width  = unit(3, "lines"),
        legend.key.height  = unit(0.75, "lines"),
        legend.position = "bottom") +
  guides(fill = guide_colorbar(title.position="top", title.hjust=0.5))

# ggsave("../plots/measure_maps/Arterial_density_cortex.svg", width=5, height=3, units='in', dpi=300)


### Venous density in brain ###
regional_vascular_data %>%
  ungroup() %>%
  mutate(label = glue("lh_{Base_Region}")) %>%
  left_join(., as_tibble(dk)) %>%
  ggseg(atlas = dk, mapping = aes(fill = Venous_Vascular),
        position = "stacked", colour = "black", hemisphere="left",
        linewidth=0.5) +
  theme_void() +
  labs(fill = "Mean venous density") +
  scale_fill_gradient(low = "white", high = "#d41d56", na.value="white") +
  theme(plot.title = element_blank(),
        legend.key.width  = unit(3, "lines"),
        legend.key.height  = unit(0.75, "lines"),
        legend.position = "bottom") +
  guides(fill = guide_colorbar(title.position="top", title.hjust=0.5))

# ggsave("../plots/measure_maps/Venous_density_cortex.svg", width=5, height=3, units='in', dpi=300)

Joining with `by = join_by(label)`
Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
In addition: Warning messages:
1: Some data not merged properly. Check for naming errors in data:
  atlas type  hemi  side  region label       roi   Base_Region Arterial_Vascular
  <chr> <chr> <chr> <chr> <chr>  <glue>      <chr> <chr>                   <dbl>
1 <NA>  <NA>  <NA>  <NA>  <NA>   lh_accumbe… <NA>  accumbensa…              41.7
2 <NA>  <NA>  <NA>  <NA>  <NA>   lh_amygdala <NA>  amygdala                 22.4
3 <NA>  <NA>  <NA>  <NA>  <NA>   lh_brainst… <NA>  brainstem                46.9
4 <NA>  <NA>  <NA>  <NA>  <NA>   lh_caudate  <NA>  caudate                  14.9
5 <NA>  <NA>  <NA>  <NA>  <NA>   lh_hippoca… <NA>  hippocampus              21.0
6 <NA>  <NA>  <NA>  <NA>  <NA>   lh_pallidum <NA>  pallidum                 13.4
7 <NA>  <NA>  <NA>  <NA>  <NA>   lh_putamen  <NA>  putamen                  18.4
8 <NA

### Is homotopic connectivity correlated with vascular innervation?

In [16]:
HoFC_and_vascular_info = pd.merge(pearson_homotopic_res_mean, regional_vascular_data, on='Base_Region', how='left')

HoFC_and_vascular_info.head()

,Base_Region,Mean_Pearson_R,Arterial_Vascular,Venous_Vascular
0,bankssts,0.314670,5.629985,39.885609
1,caudalanteriorcingulate,0.213912,20.057773,38.279240
2,caudalmiddlefrontal,0.330858,17.089778,32.134382
3,cuneus,0.644840,15.818715,69.034659
4,entorhinal,0.029524,27.140752,80.060211


In [30]:
%%R -i HoFC_and_vascular_info -w 800 -h 400

arterial_scatter_plot <- HoFC_and_vascular_info %>% 
    ggplot(data=., mapping=aes(x=Arterial_Vascular, y=Mean_Pearson_R, fill=Arterial_Vascular)) +
    geom_point(color='black', shape=21, size=3) +
    scale_fill_gradient(low = "white", high = "#e24528", na.value="white") +
    geom_smooth(method="lm", se=FALSE, color='black') +
    ggpubr::stat_cor(method="spearman", cor.coef.name="rho", size=5, aes(label = ..r.label..)) +
    xlab("Mean Arterial Density") +
    ylab("Mean Pearson R") +
    theme(legend.position="none",
          strip.background = element_blank(),
          strip.text = element_text(face='bold', size=14))

venous_scatter_plot <- HoFC_and_vascular_info %>% 
    ggplot(data=., mapping=aes(x=Venous_Vascular, y=Mean_Pearson_R, fill=Venous_Vascular)) +
    geom_point(color='black', shape=21, size=3) +
    scale_fill_gradient(low = "white", high = "#d41d56", na.value="white") +
    geom_smooth(method="lm", se=FALSE, color='black') +
    ggpubr::stat_cor(method="spearman", cor.coef.name="rho", size=5, aes(label = ..r.label..)) +
    xlab("Mean Arterial Density") +
    ylab("Mean Pearson R") +
    theme(legend.position="none",
          strip.background = element_blank(),
          strip.text = element_text(face='bold', size=14))

wrap_plots(list(arterial_scatter_plot, venous_scatter_plot), ncol=1)

ggsave("../plots/measure_maps/Mean_Pearson_R_magnitude_vs_vascular_density_scatter.svg", width=2.7, height=5, dpi=300, units="in")

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In addition: Warning messages:
1: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
2: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
3: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
4: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping